In [17]:
# Imports
import pyrealsense2 as rs
import numpy as np
import cv2
import os
from pathlib import Path

In [18]:
# input and output paths
bag_file = Path("Sources") / "LapockaVideok" / "20251110_143017.bag"
output_dir = Path("dataset_output") / "session_01"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir / "rgb", exist_ok=True)
os.makedirs(output_dir / "depth", exist_ok=True)

In [22]:
# RealSense pipeline setup
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(str(bag_file), repeat_playback=False)
config.enable_stream(rs.stream.color, 848, 480, rs.format.rgb8, 15)
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 15)
profile = pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)


In [23]:
# Video trimming parameters

START_TIME = 4.956
END_TIME = 14.0

# --- playback object for timestamp control ---
device = profile.get_device()
playback = device.as_playback()
playback.set_real_time(False)  # ensures deterministic playback

In [24]:
# Frame capture, alignment and saving (trimmed)
frame_count = 0  # counts only saved frames
start_timestamp = None  # timestamp (s) of first received color frame

print(f"Trimming: keeping frames from {START_TIME:.3f}s to {END_TIME:.3f}s")

try:
    while True:
        frames = pipeline.wait_for_frames()
        # Align depth to color
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()

        if not color_frame or not depth_frame:
            continue

        # Use color frame timestamp for consistent timing (ms -> s)
        ts_sec = color_frame.get_timestamp() / 1000.0
        if start_timestamp is None:
            start_timestamp = ts_sec  # reference start of recording
        rel_time = ts_sec - start_timestamp

        # Skip until we reach START_TIME
        if rel_time < START_TIME:
            continue

        # Stop after END_TIME
        if rel_time > END_TIME:
            break

        # Convert frames to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())

        # Depth scaling for visualization
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET
        )

        # Save only trimmed frames (sequentially numbered from 0)
        cv2.imwrite(f"{output_dir}/rgb/frame_{frame_count:05d}.png", color_image)
        np.save(f"{output_dir}/depth/frame_{frame_count:05d}.npy", depth_image)
        cv2.imwrite(f"{output_dir}/depth/frame_{frame_count:05d}_colormap.png", depth_colormap)

        frame_count += 1

except RuntimeError:
    print("Reached end of .bag file or stream error")

pipeline.stop()
print(f"Exported {frame_count} trimmed frames between {START_TIME}s and {END_TIME}s (relative)")

Trimming: keeping frames from 4.956s to 14.000s


Trimming: keeping frames from 4.956s to 14.000s


FileNotFoundError: [Errno 2] No such file or directory: 'dataset_output/depth/frame_00000.npy'

In [ ]:
# Cropping region of interest and saving

roi = (100, 80, 600, 320)  # (x, y, w, h)
for i in range(frame_count):
    rgb = cv2.imread(f"{output_dir}/rgb/frame_{i:05d}.png")
    depth = np.load(f"{output_dir}/depth/frame_{i:05d}.npy")

    rgb_crop = rgb[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
    depth_crop = depth[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]

    cv2.imwrite(f"{output_dir}/rgb/frame_{i:05d}_crop.png", rgb_crop)
    np.save(f"{output_dir}/depth/frame_{i:05d}_crop.npy", depth_crop)